In [1]:
import time
from typing import Tuple
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, DWaveCliqueSampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit_optimization.converters import LinearEqualityToPenalty
from utilities.helpers import create_quadratic_programs_from_paths
DIR = 'TEST_DATA' + "/" + '27_04_2021'

In [2]:
# select linear programs to solve
qps = create_quadratic_programs_from_paths(DIR + "/DENSE/")
qp_dense = qps['test_70']

qps = create_quadratic_programs_from_paths(DIR + "/SPARSE/")
qp_sparse = qps['test_100']

# init solvers
cplex = CplexOptimizer()
dwave_dense = DWaveCliqueSampler()
dwave_sparse = EmbeddingComposite(DWaveSampler())
# solve classically
for qp in (qp_dense, qp_sparse):
    print(cplex.solve(qp))

optimal function value: -803.0000000000003
optimal value: [1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 0.]
status: SUCCESS
optimal function value: -1670.0
optimal value: [1. 1. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1.
 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1.
 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 0.
 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1.
 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1

In [3]:
def qp_to_bqm(qp: QuadraticProgram) -> dimod.BQM:
    conv = LinearEqualityToPenalty()
    qubo = conv.convert(qp)
    bqm = dimod.as_bqm(qubo.objective.linear.to_array(), qubo.objective.quadratic.to_array(), dimod.BINARY)
    return bqm

solver_parameters = {'num_reads': 1024, 'annealing_time': 3}

def solve_bqm(sampler: dimod.Sampler, bqm: dimod.BQM):
    sampleset = sampler.sample(bqm, **solver_parameters)
    return sampleset

In [5]:
qubo_dense = qp_to_bqm(qp_dense)
sampleset_dense = solve_bqm(dwave_dense, qubo_dense)
# 44d0d592-9107-4661-a6df-3e65836c1bda
sampleset_dense.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,chain_break_fraction,energy,num_occurrences
0,1,1,1,1,0,1,1,1,1,0,...,0,1,0,1,1,1,0,0.000000,-803.0,24
1,1,1,1,1,0,1,1,1,1,0,...,0,1,0,1,1,1,0,0.000000,-800.0,12
2,1,1,1,1,0,1,1,1,1,0,...,0,1,0,0,1,1,0,0.000000,-800.0,9
3,1,1,1,1,0,1,1,1,1,0,...,0,1,0,0,1,1,0,0.000000,-800.0,5
4,1,1,1,1,0,1,1,1,1,0,...,0,1,0,0,1,1,0,0.000000,-800.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,1,1,1,1,0,1,1,1,1,0,...,0,1,0,0,0,1,0,0.042857,-720.0,1
699,1,1,1,1,0,1,1,1,1,0,...,0,1,0,0,1,1,0,0.042857,-768.0,1
700,1,1,1,1,1,1,1,1,0,0,...,0,1,0,0,1,1,1,0.042857,-708.0,1
701,1,1,1,1,1,1,1,1,1,0,...,0,1,0,0,1,1,1,0.071429,-736.0,1


In [12]:
def eval_bqm_sampleset(qp: QuadraticProgram, sampleset) -> Tuple[bool, int, float]:
    x = sampleset.record.sample[0]
    feasible = qp.is_feasible(x)
    num_occur = sampleset.record.num_occurrences[0]
    energy = sampleset.record.energy[0]
    return feasible, num_occur, energy

In [13]:
res = eval_bqm_sampleset(qp_dense, sampleset_dense)
res

(True, 24, -803.0)